# Pytorch

![pytorch](img/pytorch.png)


In [53]:
import numpy as np

In [52]:
import torch
from torch import nn
import torch.optim as optim

In [54]:
np.random.randn(10, 2)

array([[-0.14071872, -1.73487554],
       [-0.85170154,  0.98727216],
       [ 0.16762812,  0.43609718],
       [ 1.38676303, -0.46687341],
       [-1.28889297,  0.57948545],
       [ 1.16824409,  0.27015179],
       [ 0.16671552, -0.80818652],
       [ 0.29665347,  0.16198606],
       [-1.35441872,  0.11755796],
       [-0.4238943 , -0.29595254]])

In [55]:
torch.randn(10, 2)

tensor([[ 0.0807,  0.5361],
        [-0.3592,  0.6331],
        [-1.9988, -2.3001],
        [-0.8135,  0.6358],
        [ 0.4400,  0.7665],
        [-0.5996,  1.7329],
        [ 0.1236,  0.1532],
        [ 0.5359, -3.0259],
        [ 0.6837, -0.7039],
        [-0.4395, -1.3465]])

In [19]:
class SimpleRecurrentNetwork(nn.Module):
    def __init__(self, input_size, output_size, embedding_size=128, hidden_size=128):
        super(SimpleRecurrentNetwork, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
    
        self.emb = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, num_layers=1)
        self.relu = nn.ReLU()
        
        self.linear = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.emb(x)
        _, x = self.gru(x)
        x = self.relu(x.squeeze(0))
        x = self.linear(x)
        return x

In [56]:
model = SimpleRecurrentNetwork(input_size=128, output_size=2)

In [57]:
model

SimpleRecurrentNetwork(
  (emb): Embedding(128, 128)
  (gru): GRU(128, 128)
  (relu): ReLU()
  (linear): Linear(in_features=128, out_features=2, bias=True)
)

# Example


data: https://ml-challenge.mercadolibre.com/downloads

In [58]:
import pandas as pd

In [59]:
df = pd.read_csv('data/train.csv')

In [60]:
df.head()

,title,label_quality,language,category
0,Projeto Unidade Hidraulica 3000 Psi,reliable,portuguese,AIR_COMPRESSORS
1,Tapete Capacho 120x60 Churrasqueira + Frete Gr...,reliable,portuguese,CARPETS
2,Camiseta Raglan Crepúsculo Jealous Baby Look,reliable,portuguese,T_SHIRTS
3,Unidade De Dvd Gravador Com Defeito Apenas Par...,reliable,portuguese,DVD_RECORDERS
4,Fan Dell R320 / R420 0hr6c0 - 24h,reliable,portuguese,DESKTOP_COMPUTER_COOLERS_AND_FANS


In [61]:
from torchtext import data
from torch.utils.data import DataLoader

In [62]:
field_x, field_y = data.Field(), data.LabelField()
dataset = data.TabularDataset(path='data/train.csv', format='csv',
                              fields=[('title', field_x), 
                                      ('label_quality', None),
                                      ('language', None),
                                      ('category', field_y)])

In [63]:
train, valid = dataset.split()

In [64]:
train_iter, val_iter = data.BucketIterator.splits(
    (train, valid), batch_sizes=(16, 16),
    sort_key=lambda x: len(x.title))

In [65]:
field_x.build_vocab(dataset, min_freq=10)
field_y.build_vocab(dataset)

In [66]:
criterion = nn.CrossEntropyLoss()

model = SimpleRecurrentNetwork(input_size=len(field_x.vocab), output_size=len(field_y.vocab))
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [67]:
len(field_x.vocab), len(field_y.vocab)

(1321, 878)

In [68]:
total_iters = len(train_iter)

for e in range(1):
    model.train()
    train_loss = 0
    for i, batch in enumerate(train_iter):
        optimizer.zero_grad()
        x, y = batch.title, batch.category
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if i % 10 == 0:
            print(f'Iteration: {i}/{total_iters}, Train Loss: {train_loss / (i + 1):.4f}')

    model.eval()
    with torch.no_grad():
        eval_loss = 0
        for batch in val_iter:
            x, y = batch.title, batch.category
            output = model(x)
            loss = criterion(output, y)
            eval_loss += loss.item()
        print(f'Epoch: {e:3}, Evaluatio Loss: {eval_loss:.4f}')

Iteration: 0/438, Train Loss: 6.7495
Iteration: 10/438, Train Loss: 6.8218
Iteration: 20/438, Train Loss: 6.7216
Iteration: 30/438, Train Loss: 6.6746
Iteration: 40/438, Train Loss: 6.6425
Iteration: 50/438, Train Loss: 6.6148
Iteration: 60/438, Train Loss: 6.5658
Iteration: 70/438, Train Loss: 6.5481
Iteration: 80/438, Train Loss: 6.5233
Iteration: 90/438, Train Loss: 6.5044
Iteration: 100/438, Train Loss: 6.4769
Iteration: 110/438, Train Loss: 6.4428
Iteration: 120/438, Train Loss: 6.4052
Iteration: 130/438, Train Loss: 6.3756
Iteration: 140/438, Train Loss: 6.3454
Iteration: 150/438, Train Loss: 6.3252
Iteration: 160/438, Train Loss: 6.2997
Iteration: 170/438, Train Loss: 6.2633
Iteration: 180/438, Train Loss: 6.2294
Iteration: 190/438, Train Loss: 6.1931
Iteration: 200/438, Train Loss: 6.1582
Iteration: 210/438, Train Loss: 6.1286
Iteration: 220/438, Train Loss: 6.1085
Iteration: 230/438, Train Loss: 6.0684
Iteration: 240/438, Train Loss: 6.0303
Iteration: 250/438, Train Loss: 5.98

In [69]:
from sklearn import metrics

In [70]:
metrics.accuracy_score(y.numpy(), torch.argmax(output, dim=1).numpy())

0.25